In [ ]:
source(here::here("R/setup.R"))
df_hazard <- 
  df_hazard %>% 
  st_drop_geometry()

# カテゴリデータの取り扱い

ダミー変数の作成

カテゴリデータが表現する値は連続的に変化するものではありません。

多くの統計・機械学習モデルでは、数値化を求めます。

だめ... SVM、ニューラルネットワーク

xgboost, glmnet etc.

カテゴリの変数には、次にあげる特徴が含まれる場合があります。

1. 大小または順序関係
2. 重み

ここで紹介する多くの特徴量エンジニアリングは、カテゴリ変数がもつ特徴を考慮しつつ、数値化するものとなります。

定性的

一方で、数値のように扱える郵便番号などは数値として扱ってはいけません。これらは数値出会っても大小関係や連続的な意味をもたないためです。

「どれだけ違うか」ではなく「値が異なることが重要」

メッシュコード

尺度の問題？？

カテゴリに順序を与える
大きさを示す変数として「大」、「中」、「小」の3項目がある場合、「大」は「小」よりも大きいことはわかります。この関係は1から3の数値に示すことが可能で、大は一番大きな値である3と対応するという変換を行うことができます。

<!-- ここではビールへの支出データおよび土砂災害・雪崩メッシュデータを利用します。 -->

In [ ]:
library(cattonum)

In [ ]:
df_beer2018q2
df_hazard
df_lp_kanto

カテゴリ変数の特徴量エンジニアリングには、次元を増やす、増やさないの両方のパターンがあります。

それぞれの方法をみていきましょう。

色々ある。これで全てではない。weight of evidenceなど。

カテゴリを数値に変換する処理のことを全般的にエンコーディング

## ビンカウンティング

統計量を当てはめるのをビンカウンティング

### カウントエンコーディング

カウント変数の項目に対して、頻度を求めたものがカウントエンコーディングです。

In [ ]:
set.seed(1236)
df <- 
  df_hazard %>% 
  sample_n(10) %>% 
  select(hazardDate, hazardType, maxRainfall_h)
df

このようなデータに対して、hazardTypeのカウントエンコーディングを適用すると次のようになります。

In [ ]:
df %>% 
  group_by(hazardType) %>% 
  mutate(hazardDate,
            hazardType_ = n(),
            maxRainfall_h) %>% 
  ungroup() %>% 
  select(hazardDate, hazardType = hazardType_, maxRainfall_h)

カテゴリ内での出現頻度が多ければ多いほど、特徴量の値は大きくなり、影響も強くなります。一方で、元は異なる水準であったものが同じ出現頻度であった場合にはエンコード後の値が同じになってしまうことに注意です。例では、出現頻度が1の「地すべり」と「雪崩」

### ラベルエンコーディング

ラベルエンコーディング (label encoding) はカテゴリに対して一意の数値を割り振るというアイデアが単純なものですが、それではカテゴリがもつ特徴を拾い上げることはできません。

In [ ]:
df_hazard %>% 
  st_drop_geometry() %>% 
  group_by(hazardType) %>% 
  slice(1:2L) %>% 
  ungroup() %>% 
  distinct(hazardType, hazardType_sub, .keep_all = TRUE) %>% 
  select(hazardType) %>% 
  mutate(hazardType_num = as.numeric(as.factor(hazardType))) %>% 
  head(10)

### ターゲットエンコーディング

ターゲットエンコーディング (target-based encoding, likelihood encoding) は、カテゴリ変数と対応する目的変数の値を利用した方法です。カテゴリ変数の水準ごとに、水準の項目を目的変数の平均値に置き換えるという処理を行います。例えば、カテゴリ変数にAという項目が4つ含まれ、それぞれに1.5, 3.0, 0, 1.2のoutcomeが与えられているとします。この場合、outcomeの平均値は1.425なので、カテゴリ変数のAは1.425に置き換えられます。また以下のように目的変数が論理値である場合には、それを数値に変換した値を利用します（RではTRUEが1、FALSEが0）。

In [ ]:
df <- 
  tibble(
  feature = c("A", "A", "A", "A", "B", "B", "C", "C"),
  outcome = c(TRUE, FALSE, FALSE, FALSE, TRUE, FALSE, TRUE, TRUE)) %>% 
  add_count(feature)

df

df %>% 
  mutate(outcome = as.numeric(outcome)) %>% 
  catto_mean(response = outcome)

# df_target_enc <- 
#   df %>% 
#   add_count(feature) %>%
#   group_by(feature) %>% 
#   mutate(mean_encode = sum(outcome) / n) %>% 
#   ungroup() %>% 
#   select(-feature)
# 
# df_target_enc

データリークを起こしてしまう問題がある。

また

頻度の低い水準がある場合も過学習の原因になってしまう可能性がある。

#### Leave one out エンコーディング

ターゲットエンコーディングの計算において、

データリークを防ぐように

自身を除いて計算します。

完全に防げるわけではない?

In [ ]:
# df_target_enc %>% 
#   group_by(feature) %>% 
#   mutate(loo_encode = lead(outcome))

df %>% 
  mutate(outcome = as.numeric(outcome)) %>% 
  catto_loo(response = outcome)

### CatBoost

## ダミー変数化

カテゴリ変数を数値に変換する処理として最も一般的なのが、カテゴリ変数をダミー変数化してしまうことです。カテゴリに含まれる水準の値を特徴量に直接用いるもので、ダミーコーディング、one-hotエンコーディング、effectコーディングの3種類があります。これらはカテゴリに含まれるk種類の値を特徴量として扱う際の挙動が異なります。

### ダミーコーディング

ダミーコーディングは統計分析でも広く使われるカテゴリ変数の数値化手法です。該当する値を含む場合に1、そうでなければ0を各特徴量に与えます。ダミーコーディングではカテゴリが取りうる数、自由度 k-1の特徴量を生成します。自由度 k-1 で十分である理由は、他のダミー変数の値から残りの一つの値が推測可能だからです。

具体例で示しましょう。3つの水準 (A, B, C)をもつカテゴリ変数をダミーコーディングすると、2つの特徴量ができます。ここでは`feature_B`,`feature_C`という名前をつけました。ここで、Aをもつデータを探すのは簡単です。ダミー変数には0と1の値が格納され、該当しない場合には0ですので`feature_B`,`feature_C`両方で0のデータがAになります。Aのようなダミー変数に含まれないカテゴリは参照カテゴリと呼ばれます。参照カテゴリに対して、`feature_B`、`feature_C`の値が決まります。

In [ ]:
df %>% 
  recipe(~ feature) %>% 
  step_dummy(feature) %>% 
  prep() %>% 
  juice()

ダミーコーディングを利用したモデリングはその結果の解釈が容易になります。これを地価公示データの都市計画区分 (`urban_planning_area`) をダミー変数化することで示しましょう。都市計画区分の列は次に示すように4つの値を取りますが、1つを参照カテゴリとして扱い、3つのダミー変数で表現することになります。

In [ ]:
unique(df_lp_kanto$urban_planning_area)

In [ ]:
df_lp_kanto_dummy_baked <- 
  df_lp_kanto %>% 
  recipe(posted_land_price ~ .) %>% 
  step_dummy(urban_planning_area) %>% 
  prep() %>% 
  bake(posted_land_price, starts_with("urban_planning_area"), new_data = df_lp_kanto)

df_lp_kanto_dummy_baked

都市計画区分の情報のみを使って、公示価格を予測する線形回帰モデルを適用します。

In [ ]:
df_lp_kanto_dummy_baked %>% 
  lm(posted_land_price ~ ., data = .) %>% 
  tidy()

推定された結果の切片は、参照カテゴリの平均値を示します。つまり「市街化」の効果です。市街化に対して、他の係数はいずれも負値を取っています。これは市街化の影響が地下価格に影響し、他のカテゴリは効果が小さいことを示す結果です。ダミーエンコーディングではカテゴリの水準の一つを切片として利用可能なため、モデルの解釈が容易になるのです。

In [ ]:
df_lp_kanto %>% 
  ggplot(aes(urban_planning_area, posted_land_price)) +
  geom_bar(stat = "identity")

フルランク未満のエンコーディングは One-hotエンコーディング

- ダミー変数が多くなると次元の数が増える (データ件数を上回ることも)




### One-hotエンコーディング

カテゴリ変数に含まれる項目を新たな列として扱い、各列の値には0または1を与えていく方法をOne-hotエンコーディングと言います。

カテゴリに該当する場合は1、そうでない場合には0を与えていく方法です（ある要素が1で他の要素が0であるようにする表現をone-hot表現と呼びます）。

ダミー変数とは異なり、ターゲットの項目も残るのが特徴です。

この変数に対してOne-hotエンコーディングを行うと

In [ ]:
df_lp_kanto$urban_planning_area %>% unique()

In [ ]:
df_lp_kanto %>% 
  recipe(~ .) %>% 
  step_dummy(urban_planning_area, one_hot = TRUE) %>% 
  prep() %>% 
  bake(starts_with("urban_planning_area"), new_data = df_lp_kanto)

今回はカテゴリの水準数が4であったために4つの特徴量が新たに作られました。

In [ ]:
df_baked_split_date %>% 
  recipe(expense ~ .) %>% 
  step_dummy(date_dow, one_hot = TRUE) %>% 
  prep(training = df_baked_split_date) %>% 
  bake(new_data = df_baked_split_date) %>% 
  select(starts_with("date_dow"), everything())

> カテゴリ数に応じて列数が増えることや、新しい値が出現する度に列数を増やす必要があることが問題点

ダミーコーディングでは、カテゴリが多い場合にデータサイズが増大し、さらに大量の0と一部の1を含んだスパースデータ (sparse data) になりやすいことに留意しましょう。カテゴリの数が多い場合には次の特徴量ハッシュやビンカウンティングが有効です。

<!-- スパースデータは計算コストが大きくなります -->

### effectコーディング

## カテゴリ変数の縮約・拡張

### Polynomial encoding

### Expansion encoding

ビールの支出データに含まれるweatherdaytime_06_00_18_00には、「晴」や「曇」だけでなく「曇一時雨」や「雨後時々曇」といった気象に関する項目が含まれます。項目の組み合わせによる表現が可能であるため、カテゴリの数は多くなっています。

In [ ]:
df_beer2018q2 %>% 
  count(weatherdaytime_06_00_18_00)

この一次や時々によって区切ることが可能な項目

を新たな特徴量として活用するのがexpansion encodingになります。

In [ ]:
df_beer2018q2_baked <- 
  df_beer2018q2 %>% 
  select(date, expense, weatherdaytime_06_00_18_00) %>% 
  tidyr::separate(weatherdaytime_06_00_18_00, 
                  sep = "(後一時|一時|後時々|時々|後)", 
                  into = paste("weatherdaytime_06_00_18_00", 
                                c("main", "sub"),
                               sep = "_"))

df_beer2018q2_baked

複雑なカテゴリを評価するのではなく、大雑把なカテゴリとして扱いたい場合にはカテゴリの項目を減らすことが有効でしょう。

In [ ]:
df_beer2018q2_baked %>% 
  count(weatherdaytime_06_00_18_00_main,
        weatherdaytime_06_00_18_00_sub)

## 多くのカテゴリを持つ場合の処理

ビンカウンティング以外の方法を紹介します。

<!-- 次元削減の項目も参照 -->

### ターゲットエンコーディングの平滑化処理

### 特徴量ハッシング

ハッシュ関数を利用した

固定の配列に変換する

## まとめ

- カテゴリ変数はツリーベースのモデルを除いて、モデルに適用可能な状態、数値に変換する必要がある
- もっとも単純なものはカテゴリに含まれる値を独立した変数として扱うこと
    - カテゴリ内の順序を考慮するには別な方法が必要
- テキストも同様に数値化が必要。一般的には頻度の少ない単語が除外される。

## 関連項目

## 参考文献

- Max Kuhn and Kjell Johnson (2013). Applied Predictive Modeling (Springer)